In [3]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
from  matplotlib.ticker import PercentFormatter
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import ks_2samp
from scipy.stats import ttest_ind
from scipy.stats import kruskal
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

import os
import json

with open('data/paths.json','r') as f:
    paths = json.load(f)

data_path = paths["data_path"]

<h1 style="color: red">Analysis of Brain Tissue Lipids Data</h1>

<p> The study is based on the following process:: we integrate the chromatographic peak for each lipid species to obtain the “area”. The peak area of the lipid species is then normalized to the internal standard (“IS Area”) to obtain the “area ratio”.  The area ratios are then normalized to the tissue weight of the original sample.</p>
<p>
The weight normalized area ratio of each lipid species in each sample was then used to create the pivot tables and perform statistical analysis. </p>

<h2>Preprocessing data</h2>

In [16]:
# importing file inside pandas dataframe

df = pd.read_excel(os.path.join(data_path,"20220228_Laezza_DRJ_Brain Tissue_ Lipids_raw data.xlsx"), skiprows=1)

In [17]:
df.head()

,Sample Name,Sample Name on Tube,Injection Volume,Component Name,Mass Info,IS Name,Component Group Name,Expected RT,Area,IS Area,Area Ratio,Retention Time,Signal / Noise,Tissue Weight (mg),Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,10,SM(18:1)+H_d9_SPLASH.IS,738.7 / 184.2,NaN,SM,12.51,6.142877e+07,NaN,NaN,12.514791,141.426257,56,NaN,NaN,NaN,NaN
1,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,10,SM(14:0)+H,675.5 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,12.51,4.116176e+06,6.142877e+07,0.067007,12.737513,77.971430,56,0.119656,NaN,Scaling Factor:,100.0
2,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,10,SM(16:0)+H,703.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,12.51,2.433723e+07,6.142877e+07,0.396186,12.662914,147.602022,56,0.707475,NaN,NaN,NaN
3,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,10,SM(18:0)+H,731.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,12.51,1.432478e+08,6.142877e+07,2.331933,12.580159,144.222304,56,4.164166,NaN,NaN,NaN
4,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,10,SM(18:1)+H,729.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,12.51,1.268323e+08,6.142877e+07,2.064706,12.572303,193.985512,56,3.686974,NaN,NaN,NaN


In [18]:
# The subject label is composed of 3 sections: treatmentCode+subjectnumber sexCode tissueCode. There could be some problems in labeling (like white spaces) 
# so we need to be sure that all the labels are consistent. 
#
# 1) We remove the white spaces 
# 2) We relabel using the format treatmentCode+subjectnumber sexCode tissueCode

def organizeSampleName(label):
    label = label.replace(" ","")
    return label[0:3] + ' ' + label[3:5] + ' ' + label[5:]

df['Sample Name on Tube '] = df['Sample Name on Tube '].apply(lambda x: organizeSampleName(x))
df.head()

,Sample Name,Sample Name on Tube,Injection Volume,Component Name,Mass Info,IS Name,Component Group Name,Expected RT,Area,IS Area,Area Ratio,Retention Time,Signal / Noise,Tissue Weight (mg),Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,10,SM(18:1)+H_d9_SPLASH.IS,738.7 / 184.2,NaN,SM,12.51,6.142877e+07,NaN,NaN,12.514791,141.426257,56,NaN,NaN,NaN,NaN
1,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,10,SM(14:0)+H,675.5 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,12.51,4.116176e+06,6.142877e+07,0.067007,12.737513,77.971430,56,0.119656,NaN,Scaling Factor:,100.0
2,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,10,SM(16:0)+H,703.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,12.51,2.433723e+07,6.142877e+07,0.396186,12.662914,147.602022,56,0.707475,NaN,NaN,NaN
3,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,10,SM(18:0)+H,731.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,12.51,1.432478e+08,6.142877e+07,2.331933,12.580159,144.222304,56,4.164166,NaN,NaN,NaN
4,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,10,SM(18:1)+H,729.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,12.51,1.268323e+08,6.142877e+07,2.064706,12.572303,193.985512,56,3.686974,NaN,NaN,NaN


In [47]:
PND_30 = ['C20 M1 S','C21 M1 S','C11 M4 S','C20 F1 S','C21 F1 S','C11 F1 S','T23 M1 S','T24 M2 S','T34 M2 S','T20 F1 S','T33 F2 S','T34 F2 S','C20 M1 C','C21 M1 C',
          'C11 M2 C','C20 F1 C','C21 F1 C','C11 F1 C','T23 M1 C','T24 M2 C','T34 M2 C','T20 F1 C','T33 F2 C','T34 F2 C']

In [20]:
df['PND'] = df['Sample Name on Tube '].apply(lambda x: 30 if x in PND_30 else 60)
df.tail()

,Sample Name,Sample Name on Tube,Injection Volume,Component Name,Mass Info,IS Name,Component Group Name,Expected RT,Area,IS Area,Area Ratio,Retention Time,Signal / Noise,Tissue Weight (mg),Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,PND
57259,08312021_JDR_FL_Lipid_Sample #48,T18 F2 C,10,EPA,301.0 / 257.0,TAG(15:0/18:1/15:0)+NH4d7_SPLASH.IS,PUFA,2.65,9.494550e+05,429984.07819,2.208117,2.722196,70.891580,40,5.520292,NaN,NaN,NaN,60
57260,08312021_JDR_FL_Lipid_Sample #48,T18 F2 C,10,DHA,327.0 / 283.0,TAG(15:0/18:1/15:0)+NH4d7_SPLASH.IS,PUFA,2.82,1.502219e+08,429984.07819,349.366103,2.558918,284.097687,40,873.415258,NaN,NaN,NaN,60
57261,08312021_JDR_FL_Lipid_Sample #48,T18 F2 C,10,DPA,329.0 / 285.0,TAG(15:0/18:1/15:0)+NH4d7_SPLASH.IS,PUFA,2.80,1.058937e+07,429984.07819,24.627343,2.543228,176.422195,40,61.568359,NaN,NaN,NaN,60
57262,08312021_JDR_FL_Lipid_Sample #48,T18 F2 C,10,Adrenic Acid,331.0 / 287.0,TAG(15:0/18:1/15:0)+NH4d7_SPLASH.IS,PUFA,2.79,1.978464e+06,429984.07819,4.601250,2.563011,83.911044,40,11.503124,NaN,NaN,NaN,60
57263,08312021_JDR_FL_Lipid_Sample #48,T18 F2 C,10,AA,303.0 / 259.0,TAG(15:0/18:1/15:0)+NH4d7_SPLASH.IS,PUFA,2.81,1.463774e+08,429984.07819,340.425180,2.559217,270.128505,40,851.062950,NaN,NaN,NaN,60


In [21]:
# number of rows and columns

df.shape

(57264, 19)

In [22]:
# titles of the columns

print(df.columns.values)

['Sample Name' 'Sample Name on Tube ' 'Injection Volume' 'Component Name'
 'Mass Info' 'IS Name' 'Component Group Name' 'Expected RT' 'Area'
 'IS Area' 'Area Ratio' 'Retention Time' 'Signal / Noise'
 'Tissue Weight (mg) ' 'Unnamed: 14' 'Unnamed: 15' 'Unnamed: 16'
 'Unnamed: 17' 'PND']


In [23]:
# last column is the 'Unnamed: 14' which actually is the Normalized Area ratio (( Area ratio ÷ tissue weight)* scaling factor)

df.rename(columns={'Unnamed: 14':'Normalized Area'}, inplace=True)

In [24]:
# unique subjects

len(df['Sample Name'].unique())

48

In [25]:
# The Treatment group is identified by the first character on the "Sample Name on Tube" column: (C) for Control (T) for Deltamethrin

df.insert(2,'Treatment',['Control' if x[0] == 'C' else 'Deltamethrin' for x in df.iloc[:,1]])

In [26]:
df.head()

,Sample Name,Sample Name on Tube,Treatment,Injection Volume,Component Name,Mass Info,IS Name,Component Group Name,Expected RT,Area,IS Area,Area Ratio,Retention Time,Signal / Noise,Tissue Weight (mg),Normalized Area,Unnamed: 15,Unnamed: 16,Unnamed: 17,PND
0,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Control,10,SM(18:1)+H_d9_SPLASH.IS,738.7 / 184.2,NaN,SM,12.51,6.142877e+07,NaN,NaN,12.514791,141.426257,56,NaN,NaN,NaN,NaN,30
1,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Control,10,SM(14:0)+H,675.5 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,12.51,4.116176e+06,6.142877e+07,0.067007,12.737513,77.971430,56,0.119656,NaN,Scaling Factor:,100.0,30
2,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Control,10,SM(16:0)+H,703.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,12.51,2.433723e+07,6.142877e+07,0.396186,12.662914,147.602022,56,0.707475,NaN,NaN,NaN,30
3,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Control,10,SM(18:0)+H,731.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,12.51,1.432478e+08,6.142877e+07,2.331933,12.580159,144.222304,56,4.164166,NaN,NaN,NaN,30
4,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Control,10,SM(18:1)+H,729.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,12.51,1.268323e+08,6.142877e+07,2.064706,12.572303,193.985512,56,3.686974,NaN,NaN,NaN,30


In [27]:
# The Sex group is identified by the fifth character on the "Sample Name on Tube" column: (M) for Male (F) for Female
# If the subject labeling modality will chage, the following code must be adapted

df.insert(2,'Gender',['Male' if x[4] == 'M' else 'Female' for x in df.iloc[:,1]])

In [28]:
df.head()

,Sample Name,Sample Name on Tube,Gender,Treatment,Injection Volume,Component Name,Mass Info,IS Name,Component Group Name,Expected RT,...,IS Area,Area Ratio,Retention Time,Signal / Noise,Tissue Weight (mg),Normalized Area,Unnamed: 15,Unnamed: 16,Unnamed: 17,PND
0,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,10,SM(18:1)+H_d9_SPLASH.IS,738.7 / 184.2,NaN,SM,12.51,...,NaN,NaN,12.514791,141.426257,56,NaN,NaN,NaN,NaN,30
1,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,10,SM(14:0)+H,675.5 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,12.51,...,6.142877e+07,0.067007,12.737513,77.971430,56,0.119656,NaN,Scaling Factor:,100.0,30
2,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,10,SM(16:0)+H,703.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,12.51,...,6.142877e+07,0.396186,12.662914,147.602022,56,0.707475,NaN,NaN,NaN,30
3,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,10,SM(18:0)+H,731.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,12.51,...,6.142877e+07,2.331933,12.580159,144.222304,56,4.164166,NaN,NaN,NaN,30
4,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,10,SM(18:1)+H,729.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,12.51,...,6.142877e+07,2.064706,12.572303,193.985512,56,3.686974,NaN,NaN,NaN,30


In [29]:
# The Tissue group is identified by the last character on the "Sample Name on Tube" column: (M) for Male (F) for Female
# If the subject labeling modality will chage, the following code must be adapted

df.insert(4,'Tissue',['Cortex' if x[-1] == 'C' else 'Striatum' for x in df.iloc[:,1]])

In [30]:
df.head()

,Sample Name,Sample Name on Tube,Gender,Treatment,Tissue,Injection Volume,Component Name,Mass Info,IS Name,Component Group Name,...,IS Area,Area Ratio,Retention Time,Signal / Noise,Tissue Weight (mg),Normalized Area,Unnamed: 15,Unnamed: 16,Unnamed: 17,PND
0,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,Striatum,10,SM(18:1)+H_d9_SPLASH.IS,738.7 / 184.2,NaN,SM,...,NaN,NaN,12.514791,141.426257,56,NaN,NaN,NaN,NaN,30
1,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,Striatum,10,SM(14:0)+H,675.5 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,...,6.142877e+07,0.067007,12.737513,77.971430,56,0.119656,NaN,Scaling Factor:,100.0,30
2,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,Striatum,10,SM(16:0)+H,703.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,...,6.142877e+07,0.396186,12.662914,147.602022,56,0.707475,NaN,NaN,NaN,30
3,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,Striatum,10,SM(18:0)+H,731.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,...,6.142877e+07,2.331933,12.580159,144.222304,56,4.164166,NaN,NaN,NaN,30
4,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,Striatum,10,SM(18:1)+H,729.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,...,6.142877e+07,2.064706,12.572303,193.985512,56,3.686974,NaN,NaN,NaN,30


In [31]:
# Extract sample name
df['sample Name'] = df['Sample Name on Tube '].apply(lambda x: x[1:3])

In [32]:
# Identify normalization factor
df['is Normalization Factor'] = df['Component Name'].apply(lambda x: "_SPLASH.IS" in x)

In [33]:
# Assign normalization factor

le = LabelEncoder()
le.fit(df[['IS Name', 'Component Name']].values.flatten().ravel())
df['normFactor'] = le.transform(df['IS Name'])
df['normFactor'] = df.apply(lambda x: 
        x['normFactor'] if not x['is Normalization Factor'] else
        le.transform(np.asarray([x['Component Name']]))[0], axis=1)
df['normFactor'] = le.fit_transform(df['normFactor'])

In [36]:
# Let's check if there are NaN values in the 3 new columns. If there are NaN values the relabelling process is incorrect

print(df['Gender'].isnull().values.any())
print(df['Treatment'].isnull().values.any())
print(df['Tissue'].isnull().values.any())
print(df['PND'].isnull().values.any())

False
False
False
False


In [37]:
# There are other columns which present NaN values; 
# The column that we called 'Normalized Area' has some N/A values... 

print(df['Normalized Area'].isnull().values.sum())

25897


In [38]:
# Numbers of unique components

len(df['Component Name'].unique())

1193

In [39]:
# Number of group components

len(df['Component Group Name'].unique())

23

In [40]:
# Let's consider only rows where Normalized Area is not null

df_clean = df.dropna(subset=['Normalized Area'])
print(df_clean['Normalized Area'].isnull().values.sum())

0


In [41]:
# Numbers of unique components in the dataframe without null values

len(df_clean['Component Name'].unique())

800

In [42]:
list_of_component_groups = df_clean['Component Group Name'].unique().tolist()

In [43]:
df.head(30)

,Sample Name,Sample Name on Tube,Gender,Treatment,Tissue,Injection Volume,Component Name,Mass Info,IS Name,Component Group Name,...,Signal / Noise,Tissue Weight (mg),Normalized Area,Unnamed: 15,Unnamed: 16,Unnamed: 17,PND,sample Name,is Normalization Factor,normFactor
0,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,Striatum,10,SM(18:1)+H_d9_SPLASH.IS,738.7 / 184.2,NaN,SM,...,141.426257,56,NaN,NaN,NaN,NaN,30,20,True,10
1,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,Striatum,10,SM(14:0)+H,675.5 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,...,77.971430,56,0.119656,NaN,Scaling Factor:,100.0,30,20,False,10
2,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,Striatum,10,SM(16:0)+H,703.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,...,147.602022,56,0.707475,NaN,NaN,NaN,30,20,False,10
3,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,Striatum,10,SM(18:0)+H,731.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,...,144.222304,56,4.164166,NaN,NaN,NaN,30,20,False,10
4,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,Striatum,10,SM(18:1)+H,729.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,...,193.985512,56,3.686974,NaN,NaN,NaN,30,20,False,10
5,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,Striatum,10,SM(20:0)+H,759.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,...,33.866885,56,0.157538,NaN,NaN,NaN,30,20,False,10
6,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,Striatum,10,SM(20:1)+H,757.6 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,...,154.944122,56,0.187413,NaN,NaN,NaN,30,20,False,10
7,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,Striatum,10,SM(22:0)+H,787.7 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,...,NaN,56,NaN,NaN,NaN,NaN,30,20,False,10
8,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,Striatum,10,SM(22:1)+H,785.7 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,...,11.337270,56,0.130623,NaN,NaN,NaN,30,20,False,10
9,08312021_JDR_FL_Lipid_Sample #1,C20 M1 S,Male,Control,Striatum,10,SM(24:0)+H,815.7 / 184.1,SM(18:1)+H_d9_SPLASH.IS,SM,...,29.639917,56,0.192052,NaN,NaN,NaN,30,20,False,10


In [44]:
# Drop unnecessary columns 

df.drop(columns=['Sample Name', 'Unnamed: 15', 
    'Unnamed: 16', 'Unnamed: 17', 'IS Name'
    ], inplace=True)

In [45]:
# Change column names to camelCase
cn = df.columns
cn_camelCase = [x.title().replace('/','').replace(' ','').replace('(','').replace(')','') for x in cn]
cn_camelCase = [''.join([x[0].lower(), x[1:]]) for x in cn_camelCase]
df.columns = cn_camelCase

In [46]:
# Save dataframe to data_path

df.to_csv(os.path.join(data_path, 'clean_data.csv'), index=False)